# **Standard Headmodels with MNE Python**

(c) Mina Jamshidi Idaji (https://github.com/minajamshid), Dec 2020




---




In this tutorial, we will see how to build inverse operators with MNE Python. [This tutorial](https://mne.tools/dev/auto_tutorials/source-modeling/plot_eeg_no_mri.html#sphx-glr-auto-tutorials-source-modeling-plot-eeg-no-mri-py) is the original tutorial from MNE Python that you may consider checking out.

 Before starting, make sure you have MNE Python installed. If you use the notebook on google colab (or anywhere else) and you do not have MNE Python, use the followingline to install it:

    !pip install mne

You may get a warning, saying that *MNE is installed in '/root/.local/bin' which is not on PATH. Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.*. If so, run the follwing line:

    !cp -r /root/.local/bin /usr/local

Note that you have to restart the kernel in order to be able to import the newly installed package.

Otherwise, if you are using the Python IDE on your local machine just use `pip` to install MNE.

In [3]:
import os.path as op
import numpy as np
import matplotlib.pyplot as plt
import mne
import mne.minimum_norm as minnorm
from mne.datasets import fetch_fsaverage

Let's start with downloading the fsaverage files. MNE Python provides us with BEM solutions. The following likes of codes will download the files into a file called *mne_data* in your root directory.

In [7]:
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = op.dirname(fs_dir)
subject = 'fsaverage'

0 files missing from /root/.local/lib/python3.6/site-packages/mne/datasets/_fsaverage/root.txt in /root/mne_data/MNE-fsaverage-data
0 files missing from /root/.local/lib/python3.6/site-packages/mne/datasets/_fsaverage/bem.txt in /root/mne_data/MNE-fsaverage-data/fsaverage


We wanna build a head model with ~8000 nodes. Note that the number of vertices should be as high as it can sample the *curvature* of the brain! For this Purpose, we put the variable `_oct=6` to determine the spacing. You may vist [here](https://mne.tools/dev/generated/mne.setup_source_space.html#mne.setup_source_space) for learning about spacing.

In the cell below, I define some directories, where we will save in or read from.

In [8]:
_oct = '6'

trans_dir = op.join(subjects_dir, subject, 'bem', subject + '-trans.fif')
bem_sol_dir = op.join(subjects_dir, subject, 'bem', subject + '-5120-5120-5120-bem-sol.fif')
src_dir = op.join(subjects_dir, subject, 'bem', subject + '-oct' + _oct + '-src.fif')
fwd_dir = op.join(subjects_dir, subject, 'bem', subject + '-oct' + _oct + '-fwd.fif')
inv_op_dir = op.join(subjects_dir, subject, 'bem', subject + '-oct' + _oct + '-inv.fif')

The `trans_dir` is the path to the coregistration of the standard electrode positions that comes in mne_data. The other directories relate to BEM solution, source space (saving the nodes), forward model, and inverse operator.


As the first step, let's build the source space, i.e. the sampled vertices. The output is an MNE data type [*SourceSpaces*](https://mne.tools/dev/generated/mne.SourceSpaces.html#mne.SourceSpaces).

In [11]:
src = mne.setup_source_space(subject, spacing='oct'+_oct, subjects_dir=subjects_dir, add_dist=False)
src.save(src_dir, overwrite=True)

Setting up the source space with the following parameters:

SUBJECTS_DIR = /root/mne_data/MNE-fsaverage-data
Subject      = fsaverage
Surface      = white
Octahedron subdivision grade 6

>>> 1. Creating the source space...

Doing the octahedral vertex picking...
Loading /root/mne_data/MNE-fsaverage-data/fsaverage/surf/lh.white...
Mapping lh fsaverage -> oct (6) ...
    Triangle neighbors and vertex normals...
Loading geometry from /root/mne_data/MNE-fsaverage-data/fsaverage/surf/lh.sphere...
Setting up the triangulation for the decimated surface...
loaded lh.white 4098/163842 selected to source space (oct = 6)

Loading /root/mne_data/MNE-fsaverage-data/fsaverage/surf/rh.white...
Mapping rh fsaverage -> oct (6) ...
    Triangle neighbors and vertex normals...
Loading geometry from /root/mne_data/MNE-fsaverage-data/fsaverage/surf/rh.sphere...
Setting up the triangulation for the decimated surface...
loaded rh.white 4098/163842 selected to source space (oct = 6)

You are now one step clos

In order to get to know the structure, run the following cell. 

In [13]:
print(len(src))

2


You see that `src` is like a list with length 2. The `src[0]` is the vertices of the left hemisphere and `src[1]` the vertices of the right hemisphere. Ren the cell below to see how many nodes we have:

In [15]:
print('Number of vertices in the left hemisphere:', src[0]['vertno'].shape[0])
print('Number of vertices in the right hemisphere:', src[1]['vertno'].shape[0])

Number of vertices in the left hemisphere: 4098
Number of vertices in the right hemisphere: 4098


As said before, it is about ~8000 vertices.

Now, we will build a forward solution that account a large number of electrodes (~300). Therefore, we can use it for any electrode setting. For this purpose, we load a standard montage from MNE library, and do a trick to get the channel locations. What we do is practically build a raw structure that contains all the ~300 electrodes and then set its montage. Eventually, the info attribute, saved in `**raw_info**` is what we need for forward modelling.

In [10]:
montage = mne.channels.make_standard_montage('standard_1005')
clab = montage.ch_names
raw_info = mne.create_info(ch_names=clab, sfreq=1000, ch_types=['eeg'] * len(clab))
data1 = np.zeros((len(clab), 1))
raw_temp = mne.io.RawArray(data1, raw_info)
raw_temp.set_montage(montage)
raw_info = raw_temp.info

Creating RawArray with float64 data, n_channels=343, n_times=1
    Range : 0 ... 0 =      0.000 ...     0.000 secs
Ready.


Forward modeling is done in the cell below:

In [17]:
fwd = mne.make_forward_solution(raw_info, trans=trans_dir, src=src_dir,
                                bem=bem_sol_dir, eeg=True, mindist=5.0, n_jobs=2)
mne.write_forward_solution(fwd_dir, fwd, overwrite=True, verbose=None)

    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written


The leadfiled matrix is saved in `fwd['sol']['data']`. Check the dimensions:

In [18]:
leadfield_3D = fwd['sol']['data']
print(leadfield_3D.shape)

(343, 24588)


As you expected, the first dimension is the number of electrodes and the second one, the source space vertices. However, we saw that we have ~8000 vertices on the source space. Here we have ~24000 vertices. The reason is that the forward solution that we built in `fwd` is not restricted regarding the orientation of the dipoles located on each vertex. Therefore, each vertex has three values, coresponding to its orientation. this results in $3\times 8K=25K$ values for the leadfield.

In many cases, we use the fixed-direction soltion, i.e. we restrict the orientation of the dipoles to the direction perpendicular to the cortical surface. Let's do it and then check the leadfield:

In [19]:
fwd_fixed = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True, use_cps=True)
leadfield_normal = fwd_fixed['sol']['data']
print(leadfield_normal.shape)


    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]
(343, 8196)


No surprise that the second dimension is now equal to the number of vertices, i.e. one value per vertex.

So... Up to here, we have saved the forward solution. If you have a fixed number of electrodes for all subjects, then you may compute the inverse solution and save it. Otherwise, you have to compute it separetely for each subject. 

Inthe following, we compute the inverse solution for the whole ~300 electrode setting.

We need a covariance matrix of the noise. If you have event-related data, you can use the baseline data. Otherwise, we set it to identity matrix. However, in order to account for the data-length bias, I calculate the covariance matrix of white Gaussian with hte same number of channels and data-length as our data (here ~300 channels and I put it as 8min data with sampling rate of 250).

In [20]:
size1 = (len(clab), 8*60*250) # this should be the size of your data
data = np.random.normal(loc=0.0, scale=1.0, size=size1)
raw1 = mne.io.RawArray(data, raw.info)
noise_cov = mne.compute_raw_covariance(raw1, tmin=0, tmax=None)

Creating RawArray with float64 data, n_channels=343, n_times=120000
    Range : 0 ... 119999 =      0.000 ...   119.999 secs
Ready.
Using up to 600 segments
Number of samples used : 120000
[done]


Now we compute the inverse solution. Note that instead of raw_info, you may use the `info` attribute of your `raw` file.

In [21]:
inv_op = mne.minimum_norm.make_inverse_operator(raw_info, fwd, noise_cov,
                                                fixed=False, loose=0.2, depth=0.8)
mne.minimum_norm.write_inverse_operator(inv_op_dir, inv_op)

Converting forward solution to surface orientation
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Computing inverse operator with 343 channels.
    343 out of 343 channels remain after picking
Selected 343 channels
Creating the depth weighting matrix...
    343 EEG channels
    limit = 8197/8196 = 2.032378
    scale = 748782 exp = 0.8
Applying loose dipole orientations to surface source spaces: 0.2
Whitening the forward solution.
Computing rank from covariance with rank=None
    Using tolerance 0.00084 (2.2e-16 eps * 343 dim * 1.1e+10  max singular value)
    Estimated rank (eeg): 343
    EEG: rank 343 computed from 343 data channels with 0 projectors
    Setting small EEG eigenvalues to zero (without PCA)


<ipython-input-21-dc7f68729e43>:2: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  fixed=False, loose=0.2, depth=0.8)
<ipython-input-21-dc7f68729e43>:2: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  fixed=False, loose=0.2, depth=0.8)


Creating the source covariance matrix
Adjusting source covariance matrix.
Computing SVD of whitened and weighted lead field matrix.
    largest singular value = 11.1047
    scaling factor to adjust the trace = 6.65604e+09
Write inverse operator decomposition in /root/mne_data/MNE-fsaverage-data/fsaverage/bem/fsaverage-oct6-inv.fif...
    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written
    Writing inverse operator info...
    Writing noise covariance matrix.
    Writing source covariance matrix.
    Writing orientation priors.
    [done]


You see that we did not fix the orientation. Now let's take a look at what we computed. To do so, we first restrict the direction and the inversion method.

In [22]:
inv_method = 'eLORETA'
inv_op = minnorm.prepare_inverse_operator(inv_op, nave=1, lambda2=0.05, method=inv_method)
inv_sol, _, vertno, source_nn = minnorm.inverse._assemble_kernel(inv=inv_op, label=None, method=inv_method, pick_ori='normal')

Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    The projection vectors do not apply to these channels.
    Created the whitener using a noise covariance matrix with rank 343 (0 small eigenvalues omitted)
    Computing optimized source covariance (eLORETA)...
        Using independent orientation weights
        Fitting up to 20 iterations (this make take a while)...
        Converged on iteration 10 (3.4e-07 < 1e-06)
        Updating inverse with weighted eigen leads
[done]
    Eigenleads already weighted ... 


The variable `inv_sol` includes the inverse solution. Let's look at its shape:

In [23]:
print(inv_sol.shape)

(8196, 343)


You see that t is vertics $\times$ electrodes. One option can be that you save this operator and for each subject select the electrodes that you have. The only challenge is that you should rearrage the rows so that the arrangement is the same as your electrode sequence. Otherwise, if you have ~60 electrodes if you have the forward solution saved, it does not take time to compute it for each subject separately. 